In [ ]:
# Data Wrangling
import pandas as pd
import numpy as np

#Utility
import random

# Preprocessing & Feature Engineering
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectPercentile

# Evaluation
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from sklearn.model_selection import cross_val_score

# Optuna
import optuna
from optuna.samplers import TPESampler
from optuna import Trial

# Modeling
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor

# Visualization
from matplotlib import pyplot as plt
import seaborn as sns

%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

pd.set_option('mode.chained_assignment',  None)

In [ ]:
train = pd.read_csv(r'C:\Users\dlwks\OneDrive\바탕 화면\VSCode\DACON_CODE_REVIEW\감귤 착과량\train.csv')
test = pd.read_csv(r'C:\Users\dlwks\OneDrive\바탕 화면\VSCode\DACON_CODE_REVIEW\감귤 착과량\test.csv')

In [ ]:
from collections import Counter

def print_mode(df, col):
    cnt = Counter(df[col])
    list_cnt = cnt.most_common(3)

    for idx, value in enumerate(list_cnt):

        print(f'{col}의 최빈값 {idx + 1}순위 : {value[0]} & {value[-1]}개')

In [ ]:
def print_statistics(df, col):

    max = df['착과량(int)'].max()
    min = df['착과량(int)'].min()
    mean = df['착과량(int)'].mean()
    median = df['착과량(int)'].median()

    print(f'{col}의 최대값 : {max}')
    print(f'{col}의 최소값 : {max}')
    print(f'{col}의 평균값 : {max}')
    print(f'{col}의 중앙값 : {max}')
    
    print_mode(df, col)

In [ ]:
def identify_hist(df, col):

    sns.histplot(data = df[col], kde = True)
    print_statistics(df, col)

In [ ]:
identify_hist(train, '착과량(int)')

In [ ]:
y_train = train['착과량(int)']
X_drop_list = ['ID']
X_train = train.drop(X_drop_list, axis = 1)
X_test = test.drop(['ID'], axis = 1)

In [ ]:
high_corr = train.corr().abs().sort_values(by = '착과량(int)', ascending = False).iloc[:,:1]
features_name = high_corr[high_corr['착과량(int)'] > 0.9].index
features_name = list(features_name)
features_name.remove('착과량(int)')
X, y = X_train.drop(['착과량(int)'], axis = 1), X_train['착과량(int)']

X_1 = X[features_name]
X_test_1 = X_test[features_name]

In [ ]:
X_1 = X_1.apply(lambda x : x.clip(x.quantile(.01), x.quantile(.99)), axis = 0)